In [1]:
# !wget https://github.com/tonylampada/eleicoes22/blob/main/segundoturno/urnas.csv.zip?raw=true

# !curl 'https://cdn.tse.jus.br/estatistica/sead/odsele/eleitorado_locais_votacao/eleitorado_local_votacao_2022.zip' \
#   -H 'authority: cdn.tse.jus.br' \
#   -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' \
#   -H 'accept-language: en-US,en;q=0.9,pt;q=0.8' \
#   -H 'cookie: _ga=GA1.3.602040182.1667842424; _gid=GA1.3.1277050925.1667842424; _gat=1; TS010c81b8=0103a0ceaea1f11e895b0246e83bc9a8b7bdfca3be728637ac0025cbef5e408f0279cc520ba2c6713a92af1819fd6cefd12e41f53e' \
#   -H 'referer: https://dadosabertos.tse.jus.br/' \
#   -H 'sec-ch-ua: "Microsoft Edge";v="107", "Chromium";v="107", "Not=A?Brand";v="24"' \
#   -H 'sec-ch-ua-mobile: ?0' \
#   -H 'sec-ch-ua-platform: "Windows"' \
#   -H 'sec-fetch-dest: document' \
#   -H 'sec-fetch-mode: navigate' \
#   -H 'sec-fetch-site: same-site' \
#   -H 'sec-fetch-user: ?1' \
#   -H 'upgrade-insecure-requests: 1' \
#   -H 'user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.35' \
#   --compressed --output eleitorado_local_votacao_2022.zip

In [2]:
# !unzip urnas.csv.zip\?raw\=true
# !unzip eleitorado_local_votacao_2022.zip

In [3]:
import pandas as pd
import numpy as np
import janitor
import folium
import geopandas as gpd

### Dados:

Carregar e preparar dados de urnas e de locais de votacao/eleitorado:

In [4]:
urnas = pd.read_csv('urnas.csv').clean_names()
eleitorado = pd.read_csv('eleitorado_local_votacao_2022.csv', sep=';', encoding='latin1').clean_names()

In [5]:
urnas.modelo.value_counts()

UE2020    192691
UE2010     93757
UE2015     85729
UE2009     48245
UE2011     26889
UE2013     24672
Name: modelo, dtype: int64

In [6]:
urnas.loc[:, 'tipo_urna'] = 'nova'
urnas.loc[urnas.modelo!='UE2020', 'tipo_urna'] = 'velha'

Identificar e filtrar zonas "mistas", com pelo menos uma urna do tipo nova ou do tipo velha:

In [7]:
filtro_urnas_zonas_mistas = (urnas.groupby(['sg_uf', 'cd_municipio', 'nr_zona'])
                                  .tipo_urna.value_counts()
                                  .unstack()
                                  .fillna(0)
                                  .reset_index()
                                  .query('nova > 0 and velha > 0'))
filtro_urnas_zonas_mistas

tipo_urna,sg_uf,cd_municipio,nr_zona,nova,velha
14,AC,1392,1,453.0,15.0
15,AC,1392,9,400.0,20.0
20,AC,1538,8,56.0,17.0
160,AM,2550,1,257.0,47.0
161,AM,2550,2,218.0,38.0
...,...,...,...,...,...
6036,TO,92410,1,149.0,182.0
6037,TO,92410,34,14.0,21.0
6061,TO,93858,2,91.0,110.0
6071,TO,95192,7,62.0,62.0


In [8]:
urnas_zonas_mistas = pd.merge(urnas, filtro_urnas_zonas_mistas[['sg_uf', 'cd_municipio', 'nr_zona']], on=['sg_uf', 'cd_municipio', 'nr_zona'])

### Pre-Processamento

Agregar coordenadas latitude e longitude correspondentes a cada seção/local de votação, do segundo turno:

In [9]:
eleitorado.query('nr_turno==2', inplace=True)
urnas_zonas_mistas = pd.merge(urnas_zonas_mistas, eleitorado[['sg_uf', 'cd_municipio', 'nr_zona', 'nr_secao', 'nr_latitude', 'nr_longitude']], on=['sg_uf', 'cd_municipio', 'nr_zona', 'nr_secao'])

In [10]:
urnas_zonas_mistas[['sg_uf', 'nm_municipio', 'nr_zona']].drop_duplicates().sg_uf.value_counts()

SP    414
AM     13
RR     12
MA      9
TO      6
AC      3
AP      2
RO      2
Name: sg_uf, dtype: int64

Computar vantagem percentual de lula, conforme sugerido em https://github.com/tonylampada/eleicoes22/tree/main/segundoturno:

In [11]:
def preprocess(df):
    df = pd.merge(df, 
                  df.groupby(['sg_uf', 'nm_municipio', 'nr_zona']).tipo_urna.value_counts()
                          .unstack()
                          .reset_index()
                          .rename(columns={'nova': 'urnas_novas', 'velha': 'urnas_velhas'})
    )
    df = pd.merge(df, 
                  df.groupby(['sg_uf', 'nm_municipio', 'nr_zona', 'tipo_urna']).votos_lula.sum()
                          .unstack()
                          .reset_index()
                          .rename(columns={'nova': 'lula_novas', 'velha': 'lula_velhas'})
    )
    df = pd.merge(df, 
                  df.groupby(['sg_uf', 'nm_municipio', 'nr_zona', 'tipo_urna']).votos_bozo.sum()
                          .unstack()
                          .reset_index()
                          .rename(columns={'nova': 'bozo_novas', 'velha': 'bozo_velhas'})
    )
    df.loc[:, 'vantagem_lula_percentual'] = 100.*df.lula_velhas / (df.lula_velhas + df.bozo_velhas) - 100.*df.lula_novas / (df.lula_novas + df.bozo_novas)

    return df

In [12]:
urnas_zonas_mistas = urnas_zonas_mistas.pipe(preprocess)

In [13]:
lista_zonas_por_vantagem_lula = (
    urnas_zonas_mistas[['sg_uf', 'nm_municipio', 'nr_zona', 'vantagem_lula_percentual']].drop_duplicates()
                                                                                        .sort_values('vantagem_lula_percentual', ascending=False)
                                                                                        .reset_index(drop=True)
)

lista_zonas_por_vantagem_lula

,sg_uf,nm_municipio,nr_zona,vantagem_lula_percentual
0,AP,MACAPÁ,2,32.744549
1,RR,PACARAIMA,7,27.494545
2,RR,BONFIM,5,25.138730
3,RO,PORTO VELHO,20,24.721250
4,RR,NORMANDIA,5,23.201833
...,...,...,...,...
456,MA,CHAPADINHA,42,-12.086479
457,RO,PORTO VELHO,6,-13.080956
458,SP,BOITUVA,369,-14.225362
459,MA,IMPERATRIZ,33,-14.739729


Exportar geojson com dados espaciais:

In [14]:
geo_df = gpd.GeoDataFrame(urnas_zonas_mistas, geometry=gpd.points_from_xy(urnas_zonas_mistas.nr_longitude, urnas_zonas_mistas.nr_latitude))

geo_df.loc[geo_df.tipo_urna == 'velha', 'marker-color'] = 'red'
geo_df.loc[geo_df.tipo_urna == 'nova', 'marker-color'] = 'blue'

geo_df[['nm_municipio', 'nr_zona', 'marker-color', 'geometry']].to_file('urnas_eleicao2022.geojson', driver="GeoJSON")

### Visualização:

Variar o valor do indice `i` para checar diferentes zonas, ordenadas pela variável `vantagem_lula_percentual`:

In [15]:
i=0
print(lista_zonas_por_vantagem_lula.iloc[[i]])

gdf = pd.merge(geo_df, lista_zonas_por_vantagem_lula.iloc[[i]])
m = folium.Map(width=500,height=500, location=[gdf.centroid.y.median(), gdf.centroid.x.median()], tiles='OpenStreetMap', zoom_start=8)
for _, d in gdf.iterrows():
    m.add_child(
        folium.CircleMarker(
            location=[d.nr_latitude, d.nr_longitude],
            color='red' if d.tipo_urna == 'velha' else 'blue'
        )
    )

m

  sg_uf nm_municipio  nr_zona  vantagem_lula_percentual
0    AP       MACAPÁ        2                 32.744549
